In [1]:
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.preprocessing import normalize
from sklearn.decomposition import PCA
from sklearn.cluster import MiniBatchKMeans,MeanShift, estimate_bandwidth, SpectralClustering
from sklearn.preprocessing import normalize
%matplotlib inline
from sklearn import metrics
from sklearn.metrics import pairwise_distances
from sklearn.model_selection import train_test_split

/usr/local/lib/python3.7/site-packages/sklearn/utils/__init__.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Sequence


In [2]:
# 2014
df = pd.read_csv('/Users/admin/Documents/results.csv', error_bad_lines=False)

In [3]:
df.shape

(31984, 21)

In [4]:
pd.set_option('display.max_columns', 21)
df.head()

,10k,name,division,25k,gender,age,official,bib,genderdiv,ctz,35k,overall,pace,state,30k,5k,half,20k,country,city,40k
0,17.37,"Yamamoto, Hiroyuki",8,47.67,M,47,85.25,W1,8,NaN,71.40,8,3.27,NaN,59.18,8.02,39.72,37.65,JPN,Fukuoka,80.43
1,32.58,"Jeptoo, Rita",1,82.43,F,33,138.95,F1,1,NaN,116.37,21,5.30,NaN,99.33,16.22,69.47,65.83,KEN,Eldoret,132.10
2,16.62,"Van Dyk, Ernst F.",1,45.80,M,41,80.60,W2,1,NaN,67.42,1,3.08,NaN,56.45,7.75,38.03,36.10,RSA,Paarl,76.10
3,32.57,"Dibaba, Mare",3,82.43,F,24,140.58,F2,3,NaN,116.37,27,5.37,NaN,99.33,16.20,69.47,65.83,ETH,Shoa,132.95
4,17.12,"Hokinoue, Kota",2,46.37,M,40,81.23,W3,2,NaN,67.83,2,3.10,NaN,57.03,8.02,38.60,36.58,JPN,Nogata Fukuoka,76.72


In [5]:
df.columns

Index(['10k', 'name', 'division', '25k', 'gender', 'age', 'official', 'bib',
       'genderdiv', 'ctz', '35k', 'overall', 'pace', 'state', '30k', '5k',
       'half', '20k', 'country', 'city', '40k'],
      dtype='object')

In [6]:
df.dtypes

10k           object
name          object
division       int64
25k           object
gender        object
age            int64
official     float64
bib           object
genderdiv      int64
ctz           object
35k           object
overall        int64
pace         float64
state         object
30k           object
5k            object
half          object
20k           object
country       object
city          object
40k           object
dtype: object

In [7]:
df['state'].isnull().sum()

2576

In [8]:
#creating a binary row capturing wheelchair participants

def remove(row):
    if row['bib'].count('W') >0:
        return 1
    else:
        return 0

df['wheelchair'] = df.apply(remove, axis=1)

In [9]:
np.unique(df['wheelchair'])

array([0, 1])

In [10]:
# encode women as '1'
df['gender'] = np.where(df['gender']=='F', 1,0)

In [11]:
df.head()

,10k,name,division,25k,gender,age,official,bib,genderdiv,ctz,...,pace,state,30k,5k,half,20k,country,city,40k,wheelchair
0,17.37,"Yamamoto, Hiroyuki",8,47.67,0,47,85.25,W1,8,NaN,...,3.27,NaN,59.18,8.02,39.72,37.65,JPN,Fukuoka,80.43,1
1,32.58,"Jeptoo, Rita",1,82.43,1,33,138.95,F1,1,NaN,...,5.30,NaN,99.33,16.22,69.47,65.83,KEN,Eldoret,132.10,0
2,16.62,"Van Dyk, Ernst F.",1,45.80,0,41,80.60,W2,1,NaN,...,3.08,NaN,56.45,7.75,38.03,36.10,RSA,Paarl,76.10,1
3,32.57,"Dibaba, Mare",3,82.43,1,24,140.58,F2,3,NaN,...,5.37,NaN,99.33,16.20,69.47,65.83,ETH,Shoa,132.95,0
4,17.12,"Hokinoue, Kota",2,46.37,0,40,81.23,W3,2,NaN,...,3.10,NaN,57.03,8.02,38.60,36.58,JPN,Nogata Fukuoka,76.72,1


In [12]:
# fill nans with the most frequent values
df = df.apply(lambda x: x.fillna(x.value_counts().index[0]))

In [13]:
df['10k'] = pd.to_numeric(df['10k'], errors = 'coerce')
df['division'] = pd.to_numeric(df['division'], errors = 'coerce')
df['25k'] = pd.to_numeric(df['25k'], errors = 'coerce')
df['official'] = pd.to_numeric(df['official'], errors = 'coerce')
df['genderdiv'] = pd.to_numeric(df['genderdiv'], errors = 'coerce')
df['35k'] = pd.to_numeric(df['35k'], errors = 'coerce')
df['10k'] = pd.to_numeric(df['10k'], errors = 'coerce')
df['30k'] = pd.to_numeric(df['30k'], errors = 'coerce')
df['5k'] = pd.to_numeric(df['5k'], errors = 'coerce')
df['half'] = pd.to_numeric(df['half'], errors = 'coerce')
df['20k'] = pd.to_numeric(df['20k'], errors = 'coerce')
df['40k'] = pd.to_numeric(df['40k'], errors = 'coerce')

In [14]:
df.dtypes

10k           float64
name           object
division        int64
25k           float64
gender          int64
age             int64
official      float64
bib            object
genderdiv       int64
ctz            object
35k           float64
overall         int64
pace          float64
state          object
30k           float64
5k            float64
half          float64
20k           float64
country        object
city           object
40k           float64
wheelchair      int64
dtype: object

In [15]:
numerical =df[['10k',
    'division', 
    '25k', 
    'age',
    'official',
    'genderdiv', 
    '35k', 
    'overall',
    'pace',
    '30k', 
    '5k',
    'half', 
    '20k',  
    '40k']]


In [16]:
numerical = numerical.fillna(numerical.mean())

In [17]:
df_concat = pd.concat([numerical, df['gender'], df['wheelchair']], axis=1, join='outer', join_axes=None, ignore_index=False,
          keys=None, levels=None, names=None, verify_integrity=False,copy=True)

In [18]:
df_concat.head(20)

,10k,division,25k,age,official,genderdiv,35k,overall,pace,30k,5k,half,20k,40k,gender,wheelchair
0,17.37,8,47.67,47,85.25,8,71.40,8,3.27,59.18,8.02,39.72,37.65,80.43,0,1
1,32.58,1,82.43,33,138.95,1,116.37,21,5.30,99.33,16.22,69.47,65.83,132.10,1,0
2,16.62,1,45.80,41,80.60,1,67.42,1,3.08,56.45,7.75,38.03,36.10,76.10,0,1
3,32.57,3,82.43,24,140.58,3,116.37,27,5.37,99.33,16.20,69.47,65.83,132.95,1,0
4,17.12,2,46.37,40,81.23,2,67.83,2,3.10,57.03,8.02,38.60,36.58,76.72,0,1
5,32.58,4,82.45,29,140.68,4,116.37,28,5.37,99.33,16.22,69.47,65.83,132.95,1,0
6,17.65,4,47.67,28,84.65,4,70.23,4,3.23,58.60,8.38,39.72,37.65,79.83,0,1
7,30.48,5,76.95,29,129.83,5,107.47,5,4.97,92.52,15.17,64.85,61.62,122.80,0,0
8,17.12,3,46.37,43,81.23,3,67.83,3,3.10,57.03,8.00,38.60,36.60,76.70,0,1
9,30.48,20,77.68,31,137.83,20,112.27,20,5.27,94.78,15.15,65.23,61.78,129.83,0,0


In [19]:
df_concat.dtypes

10k           float64
division        int64
25k           float64
age             int64
official      float64
genderdiv       int64
35k           float64
overall         int64
pace          float64
30k           float64
5k            float64
half          float64
20k           float64
40k           float64
gender          int64
wheelchair      int64
dtype: object

In [20]:
# Train-test split
X_train, X_test = train_test_split(
    df_concat,
    test_size=0.5,
    random_state=42)

In [21]:
X_train.shape

(15992, 16)

In [22]:
from sklearn.preprocessing import Normalizer
normalizer = Normalizer(norm='l1')

#normalize only non-binary features
X_train_norm = normalizer.fit_transform(X_train.iloc[:, :14])
X_test_norm = normalizer.transform(X_test.iloc[:, :14])

In [23]:
numerical_X_train = pd.DataFrame(X_train_norm, columns = numerical.columns)

In [24]:
#creating a normalized train set (with binary features left binary)
X_train_concat = pd.concat([numerical_X_train, X_train['gender'], X_train['wheelchair']], axis=1, join='outer', join_axes=None, ignore_index=False,
          keys=None, levels=None, names=None, verify_integrity=False,copy=True)

In [25]:
X_train_concat.head(3)

,10k,division,25k,age,official,genderdiv,35k,overall,pace,30k,5k,half,20k,40k,gender,wheelchair
0,0.003174,0.077310,0.008019,0.003007,0.014591,0.390728,0.011697,0.452708,0.000557,0.009810,0.001592,0.006726,0.006367,0.013714,NaN,NaN
1,0.001703,0.045534,0.004342,0.001405,0.007824,0.265202,0.006318,0.646820,0.000299,0.005309,0.000834,0.003631,0.003441,0.007339,1.0,0.0
2,0.001392,0.112260,0.003498,0.000847,0.006632,0.319079,0.005257,0.533836,0.000253,0.004298,0.000722,0.002926,0.002773,0.006227,0.0,1.0


In [26]:
X_train_concat.shape

(24044, 16)

In [27]:
numerical_X_test = pd.DataFrame(X_test_norm, columns = numerical.columns)

In [28]:
X_test_concat = pd.concat([numerical_X_test, X_test['gender'],X_test['wheelchair']], axis=1, join='outer', join_axes=None, ignore_index=False,
          keys=None, levels=None, names=None, verify_integrity=False,copy=True)

In [29]:
X_train_concat.isnull().sum()

10k           8052
division      8052
25k           8052
age           8052
official      8052
genderdiv     8052
35k           8052
overall       8052
pace          8052
30k           8052
5k            8052
half          8052
20k           8052
40k           8052
gender        8052
wheelchair    8052
dtype: int64

In [30]:
pca = PCA(n_components=2)
X_train_pca = pca.fit_transform(X_train_concat)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [ ]:
X_test_pca = pca.transform(X_test)

# Mean Shift

In [ ]:
n_clusters = []
for sample in [X1, X2, X3, X4]:
    bandwidth = estimate_bandwidth(sample, quantile=0.2, n_samples=500)
    model = MeanShift(bandwidth=bandwidth, bin_seeding=True, n_jobs = -1)
    model.fit(sample)
    labels = model.labels_ #model.predict(test_set)
    n_clusters_ = len(np.unique(labels))
    n_clusters.append(n_clusters_)
    print(metrics.silhouette_score(sample, labels, metric='euclidean'))

In [ ]:
# the number of clusters stays consistent
n_clusters

In [ ]:
clusters = [2, 3, 5, 7, 8]
for number in clusters:
    minibatchkmeans = MiniBatchKMeans(init='k-means++',
                        n_clusters=number, 
                        random_state=42, 
                        batch_size=200).fit(X1)
    labels = minibatchkmeans.labels_
    print('Estimated number of clusters: {}'.format(number))
    print(metrics.silhouette_score(X1, labels, metric='euclidean'))

In [ ]:
clusters = [2, 3, 5, 7, 8]
for number in clusters:
    minibatchkmeans = MiniBatchKMeans(init='k-means++',
                        n_clusters=number, 
                        random_state=42, 
                        batch_size=200).fit(X2)
    labels = minibatchkmeans.labels_
    print('Estimated number of clusters: {}'.format(number))
    print(metrics.silhouette_score(X2, labels, metric='euclidean'))

In [ ]:
clusters = [2, 3, 5, 7, 8]
for number in clusters:
    minibatchkmeans = MiniBatchKMeans(init='k-means++',
                        n_clusters=number, 
                        random_state=42, 
                        batch_size=200).fit(X3)
    labels = minibatchkmeans.labels_
    print('Estimated number of clusters: {}'.format(number))
    print(metrics.silhouette_score(X3, labels, metric='euclidean'))

In [ ]:
clusters = [2, 3, 5, 7, 8]
for number in clusters:
    minibatchkmeans = MiniBatchKMeans(init='k-means++',
                        n_clusters=number, 
                        random_state=42, 
                        batch_size=200).fit(X4)
    labels = minibatchkmeans.labels_
    print('Estimated number of clusters: {}'.format(number))
    print(metrics.silhouette_score(X4, labels, metric='euclidean'))

`MiniBatchKmeans stays stable with 2 and 3 clusters.`

# Spectral Clustering

In [ ]:
clusters = [2, 3, 5, 7, 8]
for number in clusters:
    sc = SpectralClustering(n_clusters=number, n_jobs = -1)
    sc.fit(X1)
    labels = sc.labels_
    print('Estimated number of clusters: {}'.format(number))
    print(metrics.silhouette_score(X1, labels, metric='euclidean'))

In [ ]:
clusters = [2, 3, 5, 7, 8]
for number in clusters:
    sc = SpectralClustering(n_clusters=number, n_jobs = -1)
    sc.fit(X2)
    labels = sc.labels_
    print('Estimated number of clusters: {}'.format(number))
    print(metrics.silhouette_score(X2, labels, metric='euclidean'))

In [ ]:
clusters = [2, 3, 5, 7, 8]
for number in clusters:
    sc = SpectralClustering(n_clusters=number, n_jobs = -1)
    sc.fit(X3)
    labels = sc.labels_
    print('Estimated number of clusters: {}'.format(number))
    print(metrics.silhouette_score(X3, labels, metric='euclidean'))

In [ ]:
clusters = [2, 3, 5, 7, 8]
for number in clusters:
    sc = SpectralClustering(n_clusters=number, n_jobs = -1)
    sc.fit(X4)
    labels = sc.labels_
    print('Estimated number of clusters: {}'.format(number))
    print(metrics.silhouette_score(X4, labels, metric='euclidean'))

`Spectral clustering stays stable with 2 and 5 clusters.`

In [ ]:
sc = SpectralClustering(n_clusters=2, n_jobs = -1)
sc.fit(X1)
sc_labels = sc.labels_

In [ ]:
plt.figure(figsize=(20,7))

plt.subplot(141)
sc = SpectralClustering(n_clusters=2, n_jobs = -1)
sc.fit(X1)
sc_labels = sc.labels_
plt.scatter(X_pca1[:, 0], X_pca1[:, 1], c=sc_labels)
plt.title('Sample 1')

plt.subplot(142)
sc.fit(X2)
sc_2labels = sc.labels_
plt.scatter(X_pca2[:, 0], X_pca2[:, 1], c=sc_2labels)
plt.title('Sample 2')

plt.subplot(143)
sc.fit(X3)
sc_3labels = sc.labels_
plt.scatter(X_pca3[:, 0], X_pca3[:, 1], c=sc_3labels)
plt.title('Spectral Clustering With Two Clusters')
plt.title('Sample 3')

plt.subplot(144)
sc.fit(X4)
sc_4labels = sc.labels_
plt.scatter(X_pca4[:, 0], X_pca4[:, 1], c=sc_4labels)
plt.title('Sample 4')

plt.suptitle('Spectral Clustering With Two Clusters')
plt.show()

In [ ]:
minibatchkmeans = MiniBatchKMeans(init='k-means++',
                        n_clusters=2, 
                        random_state=42, 
                        batch_size=200)

In [ ]:
plt.figure(figsize=(20,7))

plt.subplot(141)

minibatchkmeans.fit(X1)
mini_labels = minibatchkmeans.labels_
plt.scatter(X_pca1[:, 0], X_pca1[:, 1], c=mini_labels)
plt.title('Sample 1')

plt.subplot(142)
minibatchkmeans.fit(X2)
mini_2labels = minibatchkmeans.labels_
plt.scatter(X_pca2[:, 0], X_pca2[:, 1], c=mini_2labels)
plt.title('Sample 2')

plt.subplot(143)
minibatchkmeans.fit(X3)
mini_3labels = minibatchkmeans.labels_
plt.scatter(X_pca3[:, 0], X_pca3[:, 1], c=mini_3labels)
plt.title('Sample 3')

plt.subplot(144)
minibatchkmeans.fit(X4)
mini_4labels = minibatchkmeans.labels_
plt.scatter(X_pca4[:, 0], X_pca4[:, 1], c=mini_4labels)
plt.title('Sample 4')

plt.suptitle('MiniBatch KMeans With Two Clusters')
plt.show()

# Centroids Location

First cluster is comprised of participants who are more likely to be older, female or in a wheelchair. They overall took almost twice as much time to reach each milestone. The second cluster is comprised of people who are more likely to be younger, male,and less likely to be disabled. They are twice as fast on average.

In [ ]:
pd.DataFrame(minibatchkmeans.cluster_centers_, columns = df_concat.columns)

In [ ]:
plt.figure(figsize=(20,7))

plt.subplot(141)
bandwidth = estimate_bandwidth(X1, quantile=0.2, n_samples=500)
ms = MeanShift(bandwidth=bandwidth, bin_seeding=True, n_jobs = -1).fit(X1)
ms_labels = ms.labels_
plt.scatter(X_pca1[:, 0], X_pca1[:, 1], c=ms_labels)
plt.title('Sample 1')

plt.subplot(142)
bandwidth = estimate_bandwidth(X2, quantile=0.2, n_samples=500)
ms = MeanShift(bandwidth=bandwidth, bin_seeding=True, n_jobs = -1).fit(X2)
ms1_labels = ms.labels_
plt.scatter(X_pca2[:, 0], X_pca2[:, 1], c=ms1_labels)
plt.title('Sample 2')

plt.subplot(143)
bandwidth = estimate_bandwidth(X3, quantile=0.2, n_samples=500)
ms = MeanShift(bandwidth=bandwidth, bin_seeding=True, n_jobs = -1).fit(X3)
ms2_labels = ms.labels_
plt.scatter(X_pca3[:, 0], X_pca3[:, 1], c=ms2_labels)
plt.title('Sample 3')

plt.subplot(144)
bandwidth = estimate_bandwidth(X4, quantile=0.2, n_samples=500)
ms = MeanShift(bandwidth=bandwidth, bin_seeding=True, n_jobs = -1).fit(X4)
ms3_labels = ms.labels_
plt.scatter(X_pca4[:, 0], X_pca4[:, 1], c=ms3_labels)
plt.title('Sample 4')

plt.suptitle('Mean Shift Clustering')
plt.show()

In [ ]:
plt.figure(figsize=(20,7))

plt.subplot(141)
sc = SpectralClustering(n_clusters=5, n_jobs = -1)
sc.fit(X1)
sc_labels = sc.labels_
plt.scatter(X_pca1[:, 0], X_pca1[:, 1], c=sc_labels)
plt.title('Sample 1')

plt.subplot(142)
sc = SpectralClustering(n_clusters=5, n_jobs = -1)
sc.fit(X2)
sc1_labels = sc.labels_
plt.scatter(X_pca2[:, 0], X_pca2[:, 1], c=sc1_labels)
plt.title('Sample 2')

plt.subplot(143)
sc = SpectralClustering(n_clusters=5, n_jobs = -1)
sc.fit(X3)
sc2_labels = sc.labels_
plt.scatter(X_pca3[:, 0], X_pca3[:, 1], c=sc2_labels)
plt.title('Sample 3')

plt.subplot(144)
sc = SpectralClustering(n_clusters=5, n_jobs = -1)
sc.fit(X4)
sc3_labels = sc.labels_
plt.scatter(X_pca4[:, 0], X_pca4[:, 1], c=sc3_labels)
plt.title('Sample 4')

plt.suptitle('Spectral Clustering with Five Clusters')
plt.show()

# Further Examination of The Clusters

In [ ]:
sc = SpectralClustering(n_clusters=5, n_jobs = -1)
sc.fit(X1)
X1['labels'] = sc.labels_

In [ ]:
# we have five unique clusters
np.unique(X1['labels'])

In [ ]:
# include all the datapoints from '0' cluster out of thethe first sample
zero = X1[X1['labels']==0]

In [ ]:

g = sns.pairplot(X1, vars = ['age','gender', 'wheelchair', 'half', '40k'], hue = 'labels')

Group zero cluster includes younger men who are not disabled and who were very fast.

In [ ]:
g = sns.pairplot(zero[['age','gender', 'wheelchair', 'half', '40k']])

In [ ]:
one = X1[X1['labels']==1]

Group one includes men and women in a wheelchair who varied in age and tended to have slower finishing results.

In [ ]:
g = sns.pairplot(one[['age','gender', 'wheelchair', 'half', '40k']])

In [ ]:
two = X1[X1['labels']==2]

Group two included younger women who were not in a wheel chair and who ran fast.

In [ ]:
g = sns.pairplot(two[['age','gender', 'wheelchair', 'half', '40k']])

In [ ]:
three = X1[X1['labels']==3]

Group three had mainly men (and some women) of various age groups who were not in a wheelchair and who tended to be slower.

In [ ]:
g = sns.pairplot(three[['age','gender', 'wheelchair', 'half', '40k']])

In [ ]:
four = X1[X1['labels']==4]

The fourth group is comprised of non-disabled men who were not as young as members of group zero and who were fast but not as fast as members of group zero or group two (young non-disabled women).

In [ ]:
g = sns.pairplot(four[['age','gender', 'wheelchair', 'half', '40k']])

# Summary

Clustering algorithms give two stable solutions for the marathon data.

`Two cluster solution divides the data into two groups (MiniBatch K-Means):` 

- Participants who are more likely to be older, female or in a wheelchair. They overall took almost twice as much time to reach each milestone. 
- People who are more likely to be younger, male,and less likely to be disabled. They are twice as fast on average.

`Five cluster solution provides the following sub-divisions (Spectral Clustering):`
- Younger men who are not disabled and who were very fast.
- Men and women in a wheelchair who varied in age and tended to have slower finishing results.
- Group two included younger women who were not in a wheel chair and who ran fast.
- Group three had mainly men (and some women) of various age groups who were not in a wheelchair and who tended to be slower.
- Non-disabled men who were not as young as members of group zero and who were fast but not as fast as members of group zero or group two (young non-disabled women).